In [ ]:
singularity pull docker://nipy/heudiconv:latest

In [ ]:
# Get scan information and a heuristic template

singularity run \
-H ~/.singularity \
-B /Shared/oleary/:/hpc \
/Users/tientong/heudiconv-latest.simg \
-d /hpc/functional/UICL/dicomdata/{subject}/{session}/*/*.dcm \
-s 5002 \
-ss 64157414 \
-f convertall \
-c none \
-o /hpc/functional/UICL/BIDS

# everything will be store in /hpc/functional/UICL/BIDS/.heudiconv

Edit the heuristic file, saved to `heudiconv/convert.py`

In [ ]:
# run heudiconv for single subject

singularity run \
-H ~/.singularity \
-B /Shared/oleary/:/hpc \
/Users/tientong/heudiconv-latest.simg \
-d /hpc/functional/UICL/dicomdata/{subject}/{session}/*/64157414*.dcm \
-s 5002 \
-ss 64157414 \
-f /hpc/functional/UICL/BIDS/code/heudiconv/convert.py `# place where I saved the edited heuristic converter` \
-c dcm2niix -b `# converter I want to use (into bids format)` \
-o /hpc/functional/UICL/BIDS

# Time 2  = 119 subjects

First get the list of subject id and session number

`/Shared/oleary/functional/UICL/BIDS/code/time2/time2_id.txt` = list of all subjects had "gender time 2" data

In [ ]:
%%bash
for sub in $(cat /Shared/oleary/functional/UICL/BIDS/code/time2/time2_id.txt | tr "\n" " ") ; do
    for i in /Shared/oleary/functional/UICL/dicomdata/$sub/[0-9]* ; do 
        ses=(`echo $i | awk '{gsub("/"," "); print $7}'`)
        echo ${sub}_${ses} >> time1time2_id.txt
    done
done

### Issue 1: Find the list of sub and ses for Time 2 data

Run this:

`/Shared/oleary/functional/UICL/BIDS/code/time2/gett2session.Rmd`

to get the session numbers of time 2, the list printed in this txt file 

`/Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt`


### Issue 2: For some subjects, ses included both corrected and uncorrected dicoms

this is post-processing done automatically. E.g., 3011 

`/Shared/oleary/functional/UICL/dicomdata/3011/63029716/T2_013/`

and

`/Shared/oleary/functional/UICL/dicomdata/3011/63029716/T2_1300/`

have to get rid of the ones with the bigger number (uncorrected).

First, get a list of all scans from all 119 people

In [ ]:
%%bash

sub=($(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))
ses=($(awk '{print $3}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))

for n in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 ; do

ls -d /Shared/oleary/functional/UICL/dicomdata/${sub[$n]}/${ses[$n]}/*/ >> /oleary/functional/UICL/BIDS/code/time2/allscanslist.txt

done

Then, run `/oleary/functional/UICL/BIDS/code/time2/move_extrascans.R` to move extra scans to `/Shared/oleary/functional/UICL/dicomdata/extra`.

# Back to Heudiconv

Create the template script below, which can be used to generate subject specific script

In [ ]:
%%bash

#!/bin/bash

subject_log=/Shared/oleary/functional/UICL/BIDS/code/time2/heudiconvT2_log/SUBJECT_SESSION.txt

echo '#--------------------------------------------------------------------------------' >> ${subject_log}

singularity run \
-H /Users/tientong/.singularity \
-B /Shared/oleary/:/hpc \
/Users/tientong/heudiconv-latest.simg \
-d /Shared/oleary/functional/UICL/dicomdata/{subject}/{session}/*/*.dcm \
-s SUBJECT \
-ss SESSION \
-f /Shared/oleary/functional/UICL/BIDS/code/heudiconv/convert.py \
-c dcm2niix -b \
-o /Shared/oleary/functional/UICL/BIDS/time2 2>&1 | tee ${subject_log}

date +"end_time: %Y-%m-%d_%H-%M-%S" >> ${subject_log}
echo '' >> ${subject_log}


use `sed` to replace placeholders with actual subject id and sessions to create subject specific script

In [ ]:
%%bash

sub=($(awk '{print $2}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))
ses=($(awk '{print $3}' /Shared/oleary/functional/UICL/BIDS/code/time2/time2_idses.txt))

for n in 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 ; do
sed -e "s|SUBJECT|${sub[$n]}|" -e "s|SESSION|${ses[$n]}|" heudiconv_TEMPLATE.sh > heudiconv_sub-${sub[$n]}_ses-${ses[$n]}.sh
done

run multiple scripts:

`ls heudiconv_sub* | xargs -i{} -P5 sh {}`

In [3]:
for number in range(0, 119):
    print(number, end=" ")
    

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 